In [1]:
# import necessary packages 
import warnings 
warnings.simplefilter('ignore',DeprecationWarning)

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

from sklearn.feature_extraction.text import CountVectorizer 
import lda 
import re


# Nlp packages  
import spacy
import nltk
nltk.download('wordnet')
import gensim
from gensim import corpora




number_topics = 3
number_words = 10

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tiam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# read in the data 
# always use pd.ExcelFile to read in an excel file so that we can extract the different workbooks inside 
data = pd.ExcelFile(r'C:/Users/Tiam/Documents/CS 791/Data/2.21.20 Logistic Epic Notes.xlsx')

In [3]:
# create the different dataframes for each of the notes
df_logician = data.parse(data.sheet_names[0])
df_epic = data.parse(data.sheet_names[1])
    

In [4]:
df_logician.head()

,ID,ENROLL,DOCNAME,CLINDATE,NOTE_TEXT,Alcohol Use,Amount,Alcohol Abuse Current,Alcohol Abuse Past,Current Drug Use,Past Drug Use,Type,Current Smoker,Former Smoker,Amount.1,Notes/Questions
0,2654.0,ENR-019366,Office Visit,2013-12-20,{{{ Arial;}}{ ; ; ; ; ; ; ; ;}{{ Normal;}{ De...,N,0,NaN,NaN,NaN,NaN,NaN,NaN,y,NONE QUIT YEARS AGO,NaN
1,2654.0,ENR-019366,Office Visit,2013-12-09,{{{ Arial;}}{ ; ; ; ; ; ; ; ;}{{ Normal;}{ De...,N,0,NaN,NaN,NaN,NaN,NaN,NaN,Y,QUIT 1970S,NaN
2,2654.0,ENR-019366,Office Visit,2014-09-03,{{{ Arial;}}{ ; ; ; ; ; ; ; ;}{{ Normal;}{ De...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2654.0,ENR-019366,Office Visit,2014-05-09,{{{ Arial;}}{ ; ; ; ; ; ; ; ;}{{ Normal;}{ De...,N,0,NaN,NaN,NaN,NaN,NaN,NaN,Y,NONE QUIT YEARS AGO,NaN
4,2654.0,ENR-019366,Emergency Report,2015-07-01,{{{ Courier New;}}{ ; ;}{{ Normal;}{ Default ...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"Says both ""former smoker"" and ""never smoker"""


In [5]:
# preprocessing 
df_logician.NOTE_TEXT = df_logician["NOTE_TEXT"].map(lambda x: re.sub("[({,\.:;?})]xxx",'',str(x)))
df_logician.NOTE_TEXT = df_logician["NOTE_TEXT"].map(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$",'',str(x)))
df_logician.NOTE_TEXT = df_logician["NOTE_TEXT"].map(lambda x: x.lower()) 
df_logician.NOTE_TEXT = df_logician["NOTE_TEXT"].map(lambda x: x.strip())


In [6]:
# preprocessing 
df_epic.NOTE = df_epic["NOTE"].map(lambda x: re.sub("[({,.:;?})xxx]",'',str(x)))
df_epic.NOTE = df_epic["NOTE"].map(lambda x: re.sub("[0-9]",'',str(x)))
df_epic.NOTE = df_epic["NOTE"].map(lambda x: x.lower()) 
# df_epic.NOTE = df_epic["NOTE"].map(lambda x: x.strip()) 


In [7]:
df_epic.loc[0,"NOTE"]

'ogress note subjective patient id  v  is a  yo female here for chief complaint patient presents with • annual eam here for annual eam not interested in quitting smoking declined c-scope depressed- has to find housing in  months and not sure how to go about it and not getting appropriate assistance according to her review of systems constitutional negative for activity change appetite change fatigue and unepected weight change hent negative eyes negative for visual disturbance respiratory negative for cough shortness of breath and wheezing cardiovascular negative for chest pain palpitations and leg swelling gastrointestinal negative for blood in stool constipation and diarrhea endocrine negative genitourinary negative for frequency musculoskeletal negative skin negative allergic/immunologic negative neurological negative for light-headedness and headaches hematological negative psychiatric/behavioral positive for dysphoric mood negative for sleep disturbance the patient is not nervous/

In [8]:
df_logician.loc[0,"NOTE_TEXT"]

'{{{  arial;}}{ ; ; ; ; ; ; ; ;}{{ normal;}{ default paragraph font;}{ definition term;}{ definition list;}{ definition;}{ h1;}{ h2;}{ h3;}{ h4;}{ h5;}{ h6;}{ address;}{ blockquote;}{ cite;}{ code;}{ emphasis;}{ hyperlink;}{ followedhyperlink;}{ keyboard;}{ preformatted;}{ z-bottom of form;}{ z-top of form;}{ sample;}{ strong;}{ typewriter;}{ variable;}{ html markup;}{ comment;}}\n . \n\n history of present illness                         \n history from:  pt\n patients primary language  english\n chief complaint:  my hands\n\n yr hx of htn dm  cad hl pt here today c/o on going radicular pain across shoulders into his fingers pt states he gets worried he should be doing " something " however he is not intersted in any surgical options \n no new weakness gait unchnaged no urinary sx\n\n\n problem list:     \n cervical radiculopathy (icd-723.4)\n special screening malignant neoplasm of prostate (icd-v76.44)\n hepatitis c, chronic (icd-070.54)\n hx of hsv ii, herpes simplex virus type ii 

In [9]:
# initialize the count vectorizer 

count_vec = CountVectorizer(max_df=.75,min_df=.25,max_features=100,stop_words='english')
count_data = count_vec.fit_transform(df_logician['NOTE_TEXT'])

In [10]:
count_vec2 = CountVectorizer(max_df=.75,min_df=.25,max_features=100,stop_words='english')
count_data2 = count_vec2.fit_transform(df_epic['NOTE'])

In [11]:
def print_topics(model,count_vectorizer,n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [12]:
# here we are going to predefine our model 
model = lda.LDA(n_topics=3,n_iter=1000)
model.fit(count_data)

model2 = lda.LDA(n_topics=3,n_iter=1000)
model2.fit(count_data2)

INFO:lda:n_documents: 392
INFO:lda:vocab_size: 100
INFO:lda:n_words: 57208
INFO:lda:n_topics: 3
INFO:lda:n_iter: 1000
C:\Users\Tiam\Anaconda3\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -322058
INFO:lda:<10> log likelihood: -276273
INFO:lda:<20> log likelihood: -270152
INFO:lda:<30> log likelihood: -266235
INFO:lda:<40> log likelihood: -263039
INFO:lda:<50> log likelihood: -261878
INFO:lda:<60> log likelihood: -261781
INFO:lda:<70> log likelihood: -261481
INFO:lda:<80> log likelihood: -261037
INFO:lda:<90> log likelihood: -261144
INFO:lda:<100> log likelihood: -260877
INFO:lda:<110> log likelihood: -260935
INFO:lda:<120> log likelihood: -260496
INFO:lda:<130> log likelihood: -260603
INFO:lda:<140> log likelihood: -260476
INFO:lda:<150> 

INFO:lda:<860> log likelihood: -773015
INFO:lda:<870> log likelihood: -772026
INFO:lda:<880> log likelihood: -773332
INFO:lda:<890> log likelihood: -772923
INFO:lda:<900> log likelihood: -773034
INFO:lda:<910> log likelihood: -772454
INFO:lda:<920> log likelihood: -772368
INFO:lda:<930> log likelihood: -773117
INFO:lda:<940> log likelihood: -772383
INFO:lda:<950> log likelihood: -773041
INFO:lda:<960> log likelihood: -772342
INFO:lda:<970> log likelihood: -773286
INFO:lda:<980> log likelihood: -773367
INFO:lda:<990> log likelihood: -772755
INFO:lda:<999> log likelihood: -773706


In [13]:
print_topics(model2,count_vec2,number_words)


Topic #0:
normal negative history pain chest kg skin past time status

Topic #1:
pt pain plan care treatment use time follow visit left

Topic #2:
mg tablet mouth daily total md day oral medications needed


### visualizing the results 

In [14]:
from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis
import os 

LDAvis_data_filepath = os.path.join('./ldavis_prepared_logician'+str())
# LDAvis_data_filepath2 = os.path.join('./ldavis_prepared_epic'+str())
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = sklearn_lda.prepare(model, count_data, count_vec)
#     LDAvis_prepared2 = sklearn_lda.prepare(model2, count_data2, count_vec2)


    with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)
        

pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_logician'+ str(number_topics) +'.html')
# pyLDAvis.save_html(LDAvis_prepared2, './ldavis_prepared_epic'+ str(number_topics) +'.html')


C:\Users\Tiam\Anaconda3\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
